In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
import sys
# np.set_printoptions(threshold=sys.maxsize)

In [3]:
nvda = yf.Ticker('NVDA')

nvda_hist = nvda.history(period='5y', interval='1d', end='2025-06-02')
nvda_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
dates = pd.to_numeric(nvda_hist.index)
dates = dates.to_numpy()

print(nvda_hist)

                                 Open        High         Low       Close  \
Date                                                                        
2020-06-24 00:00:00-04:00    9.445118    9.525105    9.114458    9.205159   
2020-06-25 00:00:00-04:00    9.325011    9.473770    9.152081    9.458819   
2020-06-26 00:00:00-04:00    9.468788    9.468788    9.095020    9.124921   
2020-06-29 00:00:00-04:00    9.139624    9.174260    8.870759    9.169774   
2020-06-30 00:00:00-04:00    9.283401    9.494953    9.236057    9.466547   
...                               ...         ...         ...         ...   
2025-06-16 00:00:00-04:00  143.350006  146.179993  143.199997  144.690002   
2025-06-17 00:00:00-04:00  144.490005  145.220001  143.779999  144.119995   
2025-06-18 00:00:00-04:00  144.009995  145.649994  143.119995  145.479996   
2025-06-20 00:00:00-04:00  145.449997  146.199997  142.649994  143.850006   
2025-06-23 00:00:00-04:00  142.500000  144.779999  142.029999  144.169998   

In [4]:
from sklearn.preprocessing import StandardScaler, Normalizer

scaler = StandardScaler()
dates_scaled = scaler.fit_transform(dates.reshape(-1, 1))


dates_norm = dates / max(dates)
for column in nvda_hist.columns:
    nvda_hist[column]/=max(nvda_hist[column])
nvda_hist.head(-5)

,Open,High,Low,Close,Volume
Date,,,,,
2020-06-24 00:00:00-04:00,0.061731,0.062213,0.061669,0.061612,0.291061
2020-06-25 00:00:00-04:00,0.060946,0.061877,0.061924,0.063310,0.243584
2020-06-26 00:00:00-04:00,0.061885,0.061845,0.061538,0.061075,0.383496
2020-06-29 00:00:00-04:00,0.059734,0.059921,0.060020,0.061375,0.221676
2020-06-30 00:00:00-04:00,0.060674,0.062016,0.062492,0.063361,0.238286
...,...,...,...,...,...
2025-06-09 00:00:00-04:00,0.935785,0.946995,0.960310,0.954582,0.119900
2025-06-10 00:00:00-04:00,0.932517,0.942358,0.957536,0.963483,0.100966
2025-06-11 00:00:00-04:00,0.945131,0.946995,0.959903,0.955987,0.108616


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_features, hidden_units, out_features):
        self.l1 = nn.Sequential(
            nn.Conv1d(in_channels=input_features, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
        )
        
